# Intermediate Python 1
This notebook provides a gentle introduction to some intermediate topics and concepts that are part of the core Python language.

## Decorators

A python decorator is typically applied to a function in Python and usually it modifies the decorated function in some way. They are typically denoted with `@` followed the name of your decorator function. A decorator function returns a function that is called instead of the decorated function.

### Built-in Decorators

The Python standard library includes some decorators that are built-in functions, usable in without any additional imports. Primarily these are applicable to methods of a particular class.

#### `@property`

In [ ]:
class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
    
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}' 
    
    @full_name.setter
    def full_name(self, full_name):
        names = full_name.split(' ')
        self.first_name = names[0]
        self.last_name = names[1]
        

In [ ]:
person = Person('John', 'Travolta')
print(person.first_name)
print(person.full_name)


In [ ]:
person.full_name = 'Bill Gates'
print(person.first_name)
print(person.last_name)

#### `@staticmethod` and `@classmethod`
Transforms functions of a class in to special methods

In [ ]:
from datetime import date

class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    def get_name(self):
        return f'{self.first_name} {self.last_name}' 
        
    @classmethod
    def from_dict(cls, details):
        return cls(details['first_name'], details['last_name'])
    
    @staticmethod
    def get_age(date_of_birth):
        return date.today() - date.fromisoformat(date_of_birth)
        

In [ ]:
details = {'last_name': 'Federer', 'city': 'Geneva', 'first_name': 'Roger', 'dob': '1981-04-26'}
person = Person.from_dict(details)

age = get_age(details['dob'])
# We can use date util to get this as a relative date in years
print(age)

### Custom Decorators

A python decorator is typically applied to a function in Python and usually it modifies the decorated function in some way. 

In most cases we want to *wrap* our decorated function. When we call the function it operates as normal, but the decorator provides a means to do something extra before or after the execution of the function.

To understand decorators we need to fully grasp the concept of functions as objects in Python.

The below defines a function that returns a given number of random numbers.

In [ ]:
import random

def get_randoms(num):
    return [random.randint(1, 101) for _ in range(num)]

In [ ]:
print(get_randoms)
print(get_randoms.__name__)

In [ ]:
get_randoms(10)

#### Timing the Function
Suppose we want to add a decorator to this function that will allow us to time how long this takes. 

We can pass this function object to another function (as parameter). We can also have that function that returns another function. In essence a decorator is a function that accepts a function (the decorated function) as a parameter and returns another function that is called instead.

To implement our timing decorator we need to write a function that,
* Records the start time of our function
* Runs the decorated function
* Calculates elapsed time and prints the result to the console
* Returns the result of the decorated function

In [ ]:
import time

def timeit(fn):
    def timed_fn(*args, **kwargs):
        start_time = time.time()
        result = fn(*args, **kwargs)
        duration = time.time() - start_time
        print(duration)
        return result
    
    return timed_fn

In [ ]:
print(timeit)

The `timeit` function accepts a function `fn` as a parameter. We define and return another function `timed_fn` that wraps `fn` and provides the additional timing behaviour.

In [ ]:
@timeit
def get_randoms(num):
    return [random.randint(1, 101) for _ in range(num)]

In [ ]:
randoms = get_randoms(100000)
randoms = get_randoms(500000)

For wrapping decorators the consensus is to use the `wraps` function in `functools`. This makes sure **special variables** are set correctly.

In [ ]:
print(get_randoms.__name__)

In [ ]:
import time
import functools

def timeit(fn):
    @functools.wraps(fn)
    def timed_fn(*args, **kwargs):
        start_time = time.time()
        result = fn(*args, **kwargs)
        duration = time.time() - start_time
        print(duration)
        return result
    return timed_fn

In [ ]:
print(get_randoms.__name__)

### Excercise
#### 1. Logging decorator

* Create a new decorator `@logged` to perform some logging (e.g `"Starting function"` and `"Ending function"`, using the `logging` module) before and after the execution of the decorated function. We should use the `info` function to create INFO level messages.
* Redefine the `get_randoms` function to have the `@logged` decorator to demonstrate the logging.

In [ ]:
### CODE HERE

#### 2. Slow down decorator
* Create a new decorator `@slow_down` that makes the decorated function sleeps for 1 second before execution. This should use `time.sleep` from the `time` module.
* Define a new function `get_random` that returns a single random number. Decorate this function with `@slow_down`. Redefine `get_randoms` to use this function so that we only generate one random number per second.

In [ ]:
### CODE HERE

### Advanced use of Decorators
There are some more advanced scenarios where we could do one of the following,
* Add mulitple decorators to a single function - e.g we might want a function that is both logged and timed
* Pass some parameters to our decorator functions - e.g the log level
* Decorate classes - we can also use decorators for class definitions

## Generators & Iterators

Generators are types of functions that allow us to create iterators.
Iterators are similar to sequence types like lists however the evaluation of the contents of the sequence is delayed until the values are requested.
In this sense the contents of the iterator are "generated".

Suppose we want to write a function to generate a sequence of square numbers. We could write a function that computes the values up front or (given we want to iterate over these values) we could write a generator function.

#### Using a list

In [ ]:
def squares(n):
    print("Starting")
    return [n*n for n in range(n)]

print(squares)
print(squares(10))

#### Using a generator

In [ ]:
def squares_gen(n):
    print("Starting")
    for n in range(n):
        yield n * n

print(squares_gen)
print(squares_gen(10))

for val in squares_gen(10):
    print val

Here we have introduced the `yield` keyword, this pauses the function and saves the local state so that it can be resumed where it left off. Python interpreter realises this is a generator function and when this function is called it will return a generator rather than executing the function.

A generator is a type of iterator. It implements the special function `__next__` which is used to get the next item in the sequence and allows us to use it in `for` loops in the same way as other sequence types.

In [ ]:
squares_list = squares(10)
print(squares_list)
squares_generator = squares_gen(10)
print(squares_generator)

In [ ]:
for square in squares_list:
    print(square)

for square in squares_generator:
    print(square)

### Generator Expressions
Much like we have list comprehensions, we can use a compact syntex for creating generators. With these kind of expressions we don't need to use `yield` instead we can just `( <genexpr> )`. The above squares generator function can be re-implemented as an expression as demonstrated below.

In [ ]:
squares_generator = (n*n for n in range(10))
print(squares_generator)

### Exercise
For the following you have the choice to use a generator function or a generator expression.

#### 1. Generating formatted strings
Given the following list of device versions, write a generator to format these tuples in to strings for example for the tuple `('iOS', 2.3)` we should return the string `"OS Type:iOs, Version:2.3"` 

In [ ]:
os_versions = [('iOS', 2.3), ('iOS', 2.7), ('iOS', 3.1), ('Android', 5.6), ('Android', 6.1)]

In [ ]:
### CODE HERE

#### 2. Implementing a filter function
Write a function that `yield`s only the elements that have a Truey value when passed to the given function. Call this function with `devices` to get a generator of OS versions where the OS type is `"iOS"` and the version is greater than `2.5`

In [ ]:
def filter_fn(iterable, fn):
    ### CODE HERE
    pass

def is_old_ios(os_version):
    ### CODE HERE
    pass